### This work is a folow-up experiment from the file 'test_text_tables_figure' 

This experiment explains how to use LangChain and Langchain's Unstructured package which can be used to get<br> the information from tables anf images in a complex pdf files

In [2]:
import os
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [3]:
# Set OpenAI API key
#os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
openai_api_key = "sk-proj-iAqcDxetWAEokMERGj0lT3BlbkFJ3VQGvXfr3eOuUpkgMf3V"

In [18]:
test_tables_figure_directory = "test-tables-figures"
os.makedirs(test_tables_figure_directory, exist_ok=True)  # Create the directory if it doesn't exist

In [19]:
#file_path = "AAR_SI data2/Death of a RAF Service Person and serious injuries during British Service Mountaineering Expedition Pakistan July 2022.pdf"

raw_pdf_elements = UnstructuredPDFLoader(
    file_path="AAR_SI data2/Death of a RAF Service Person and serious injuries during British Service Mountaineering Expedition Pakistan July 2022.pdf",
    strategy = "hi_res",
    hi_res_model_name="yolox",
    mode="elements",
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    extract_images_in_pdf=True,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir=test_tables_figure_directory
    #image_output_dir_path=output_path,
)

we can use 'from langchain_community.document_loaders import PyPDFDirectoryLoader' to load a directory for files<br>
directory_path = ("../../docs/integrations/document_loaders/example_data/layout-parser-paper.pdf")<br>
loader = PyPDFDirectoryLoader("example_data/")<br>
docs = loader.load()

In [20]:
documents = raw_pdf_elements.load()

In [21]:
len(documents)

137

In the 'raw_pdf_elements', mode was disabled. as a result, len(documents) came 1. 

In [23]:
documents[0].metadata['category']

'CompositeElement'

In [24]:
documents[21]

Document(page_content="Witness 01 Exhibit 02 Exhibit 13\n\nDeath of Wing Commander Henderson - 19 July 2022\n\n1.3.9. Arrival in Pakistan and initial activity. The Main Team arrived in Islamabad on 4 June 2022 and, following a period in Skardu, arrived at Askole (Figure 1.3.1) via road on 9 June 2022. They then trekked approximately 70km east to Broad Peak Base Camp (Figure 1.3.4), arriving on 15 June 2022. They conducted acclimatisation, consolidation training, and prepositioned food and equipment at camps higher up the mountain until 10 July 2022. During this period, the Development Team arrived in Islamabad on 9 July 2022. Whilst at Broad Peak Base Camp (4850m) there were five days of snowfall, from 11 July 2022 to 15 July 2022. This prevented the Main Team from leaving the camp. During this period, the Development Team arrived at Askole via road on 14 July 2022.\n\nWitness 01 Exhibit 02 Exhibit 13 Exhibit 16\n\n5 Source: BSME20 team member.\n\n1.3 - Page 4 of 22\n\nGFFIGIAL—SENSITI

<i>How many Categories?

In [25]:
categories = set()  # Use a set to store unique categories

# Iterate through each document and collect categories
for document in documents:
    category = document.metadata.get('category')  # Access the category
    if category:  # Check if the category exists
        categories.add(category)  # Add to the set

# Convert the set to a list for easier display (optional)
unique_categories = list(categories)

# Output the results
print(f"Number of unique categories: {len(unique_categories)}")
print("Unique categories:", unique_categories)

Number of unique categories: 2
Unique categories: ['CompositeElement', 'Table']


In [79]:
# USE unstructured's function to get the categories list
{doc.metadata["category"] for doc in documents}

{'CompositeElement', 'Table'}

In [26]:
table_elements = []
text_elements = []

for tabel and texts only.

In [ ]:
for element in raw_pdf_elements:
    if 'CompositeElement' in str(type(element)):
        text_elements.append(element)
    elif 'Table' in str(type(element)):
        table_elements.append(element)

!why it failed. need to know to use get method

In [28]:
# Iterate through the loaded documents and categorize them
for doc in documents:
    if doc.metadata.get("category") == "Table":
        table_elements.append(doc)
    else:
        text_elements.append(doc)

In [29]:
len(table_elements)

28

In [30]:
table_elements[0]

Document(page_content='Part 2 RECORD OF PROCEEDINGS Part 2.1 Diary of events Part 2.2 List of witnesses Part 2.3 Witness statements Part 2.4 List of attendees Part 2.5 List of exhibits Part 2.6 Exhibits Part 2.7 List of annexes Part 2.8 Annexes Part 2.9 inquiry Part 2.10 Master schedule REPORT Part 1 Part 1.1 Covering note and Glossary Part 1.2 Convening order & TORs Part 1.3 Narrative of events Part 1.4 Findings Part 1.5 Recommendations Schedule of matters not germane to the', metadata={'source': 'AAR_SI data2/Death of a RAF Service Person and serious injuries during British Service Mountaineering Expedition Pakistan July 2022.pdf', 'last_modified': '2024-05-22T13:44:00', 'text_as_html': '<table><thead><tr><th>Part 1</th><th>REPORT</th><th>Partt2</th><th>RECORD OF PROCEEDINGS</th></tr></thead><tbody><tr><td>Part 1.1</td><td>Covering note and</td><td>Part 2.1</td><td>Diary of events</td></tr><tr><td>Glossary</td><td></td><td>Part 2.2</td><td>List of witnesses</td></tr><tr><td>Part 1.2<

In [31]:
table_elements[0].page_content

'Part 2 RECORD OF PROCEEDINGS Part 2.1 Diary of events Part 2.2 List of witnesses Part 2.3 Witness statements Part 2.4 List of attendees Part 2.5 List of exhibits Part 2.6 Exhibits Part 2.7 List of annexes Part 2.8 Annexes Part 2.9 inquiry Part 2.10 Master schedule REPORT Part 1 Part 1.1 Covering note and Glossary Part 1.2 Convening order & TORs Part 1.3 Narrative of events Part 1.4 Findings Part 1.5 Recommendations Schedule of matters not germane to the'

In [32]:
type(table_elements[0].page_content)

str

In [33]:
#Displaying the table elements
from IPython.display import HTML, display

display(HTML(table_elements[0].metadata["text_as_html"]))

### <i>Now, Summarize the table

In [34]:
from langchain_openai import ChatOpenAI

# Step 2: Create a summary for the LLM
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, api_key=openai_api_key)

# Prepare the prompt for summarization
prompt = f"Please summarize the following table data:\n\n{table_elements[0].page_content}\n\n:"

# Get the summary from the LLM
summary_response = chat_model.invoke(prompt)
summary = summary_response.content 

print("\nSummary of the Table:")
print(summary)


Summary of the Table:
The provided data outlines the structure of a report, specifically focusing on two main sections: **Part 2: Record of Proceedings** and **Part 1: Report**. 

### Part 2: Record of Proceedings
- **2.1 Diary of events**: A chronological record of events.
- **2.2 List of witnesses**: A compilation of individuals who provided testimonies.
- **2.3 Witness statements**: Documented accounts from witnesses.
- **2.4 List of attendees**: Names of individuals present during the proceedings.
- **2.5 List of exhibits**: Items presented as evidence.
- **2.6 Exhibits**: Detailed descriptions or copies of the evidence presented.
- **2.7 List of annexes**: A catalog of supplementary materials.
- **2.8 Annexes**: Additional documents or information supporting the main report.
- **2.9 Inquiry**: The process of investigation or examination.
- **2.10 Master schedule**: An overall timeline of events related to the proceedings.

### Part 1: Report
- **1.1 Covering note and Glossary**: 

<i>Now, get all the tables and summarize

In [60]:
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, api_key=openai_api_key)

# Iterate through each table in the table_elements list
for index, table in enumerate(table_elements):
    # Extract the content of the current table
    table_content = table.page_content
    print(f"Table Data from table[{index}]:")
    print(table_content)  # Display the raw content

    # Prepare the prompt for summarization
    prompt = f"Please summarize the following table data:\n\n{table_content}"

    # Get the summary from the LLM using invoke
    summary_response = chat_model.invoke(prompt)  # Pass the prompt as a string

    # Access the content of the AIMessage response
    summary = summary_response.content  # Use the 'content' attribute

    print(f"\nSummary of Table {index}:")
    print(summary)
    print("\n" + "="*50 + "\n")  # Separator for readability

Table Data from table[0]:
Part 2 RECORD OF PROCEEDINGS Part 2.1 Diary of events Part 2.2 List of witnesses Part 2.3 Witness statements Part 2.4 List of attendees Part 2.5 List of exhibits Part 2.6 Exhibits Part 2.7 List of annexes Part 2.8 Annexes Part 2.9 inquiry Part 2.10 Master schedule REPORT Part 1 Part 1.1 Covering note and Glossary Part 1.2 Convening order & TORs Part 1.3 Narrative of events Part 1.4 Findings Part 1.5 Recommendations Schedule of matters not germane to the

Summary of Table 0:
The provided data outlines the structure of a report and its accompanying records related to an inquiry. 

**Record of Proceedings (Part 2)**:
- **Diary of Events (2.1)**: A chronological log of events.
- **List of Witnesses (2.2)**: Individuals who provided testimony.
- **Witness Statements (2.3)**: Written accounts from the witnesses.
- **List of Attendees (2.4)**: Participants present during the proceedings.
- **List of Exhibits (2.5)**: Documents or materials submitted as evidence.
- **

All good for Tables.<br>
All pdf Text are in 'CompositeElement'. There are 2 types of element in the parsed documents.<br>
<i>Now, next steps for getting summary from the images!

In [ ]:
# Step 1: List all image files in the directory

import re

pattern = re.compile(r'f.*\.(jpg|jpeg)$')
image_files = [f for f in os.listdir(test_tables_figure_directory) if pattern.match(f)]

summaries = []
for index, image_file in enumerate(image_files):
    # Construct the full path to the image
    image_path = os.path.join(test_tables_figure_directory, image_file)
    
    # Optional: Load the image if you want to display or process it
    # img = Image.open(image_path)  # Uncomment if you need to process the image

    print(f"Processing Image: {image_file}")

    # Prepare the prompt for summarization
    prompt = f"Please summarize the content of the following image: {image_file}"

    # Get the summary from the LLM using invoke
    summary_response = chat_model.invoke(prompt)  # Pass the prompt as a string

    # Access the content of the AIMessage response
    summary = summary_response.content  # Use the 'content' attribute
    summaries.append(summary)  # Store the summary

    print(f"Summary of Image {index}:")
    print(summary)
    print("\n" + "="*50 + "\n")  # Separator for readability

So, above failed. NOw, let's do it in different way with a helper function.

In [40]:
import base64
# Function to encode images
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [73]:
image_elements = []

pattern = re.compile(r'f.*\.(jpg|jpeg)$')
image_files = [f for f in os.listdir(test_tables_figure_directory) if pattern.match(f)]

for image_file in os.listdir(test_tables_figure_directory):
        image_path = os.path.join(test_tables_figure_directory, image_file)
        encoded_image = encode_image(image_path)
        image_elements.append(encoded_image)
print(len(image_elements))

58


<i> So, image_elements got 58 elements, but I have got 30 figure?!

### ! Above example NOt worked, 
ANOTHER WAY, below which is the easiest and best way to summarize:

In [81]:
from langchain_community.document_loaders import UnstructuredImageLoader
from transformers import pipeline

In [84]:
loader_for_image = UnstructuredImageLoader("figures_from_test/figure-90-27.jpg")

In [85]:
image_doc = loader_for_image.load()

In [86]:
print(f"Summerizing Image: {image_file}")

# Prepare the prompt for summarization
prompt = f"Please summarize the content of the following image: {image_doc}"

# Get the summary from the LLM using invoke
summary_response = chat_model.invoke(prompt)  # Pass the prompt as a string

# Access the content of the AIMessage response
summary = summary_response.content  # Use the 'content' attribute
#summaries.append(summary)  # Store the summary

#print(f"Summary of Image {index}:")
print(summary)

Summerizing Image: table-9-4.jpg
The content of the image appears to be related to a Total Safety Risk Matrix (TSRM), likely used for assessing safety risks. The mention of "Figure 1.4.13 -RAF" suggests that this is a specific figure from a larger document or report. Unfortunately, without seeing the actual image, I can't provide a detailed summary of its content or visual elements. If you can provide more information or describe the image, I can help further!


The iamge was not clear to human eyes also. It can assumed that the output is partially correct.

Now, takinf another example of figure which is very clear and extracted properly.

In [88]:

loader_for_image2 = UnstructuredImageLoader("figures_from_test/figure-51-15.jpg")
image2_doc = loader_for_image2.load()

In [92]:
print(f"Summerizing Image: {image_file}")

# Prepare the prompt for summarization
#prompt = f"Please explain the content of the following image in detail: {image2_doc}" # also works for detail explation of the image
prompt = f"Is the image taking about confidence: {image2_doc}"

# Get the summary from the LLM using invoke
summary_response = chat_model.invoke(prompt)  # Pass the prompt as a string

# Access the content of the AIMessage response
summary = summary_response.content  # Use the 'content' attribute
#summaries.append(summary)  # Store the summary

#print(f"Summary of Image {index}:")
print(summary)

Summerizing Image: table-9-4.jpg
Based on the text provided, it appears that the image is discussing a scale of likelihood or probability, which can relate to confidence in a statistical or decision-making context. The terms mentioned, such as "Extremely Unlikely," "Extremely Likely," and "About as likely as not," suggest that the image is depicting a framework for assessing confidence in outcomes or events based on their likelihood of occurrence. This kind of scale is often used in various fields, including statistics, risk assessment, and legal contexts, to communicate the degree of certainty or uncertainty regarding specific scenarios.


=== === The above experiement worked nicely. === ===